# A quick tutorial to RAG

In [1]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents.base import Document

import sys
sys.path.append("../../")
from src.config import Configuration
conf =  Configuration()
HUGGING_FACE_KEY = conf.load_hg_token(1)

In [2]:
# get embedding model
# MODEL = "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"
MODEL = "sentence-transformers/LaBSE"

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGING_FACE_KEY,
    model_name=MODEL)

# insert data to vector store
texts = ["Huy là lập trình viên", "John là người Mỹ", "Kiệt là kĩ sư",
        "Hiếu là họa sĩ", "Khoa là giám đốc", "Tài là người kinh doanh",
        "Ngân là một tư vấn viên", "Quang Trung là một vị hoàng đế",
        "Quang Lê là ca sĩ", "Kỳ Duyên là người dẫn chương trình",
        "Vạn Thịnh Phát là đại gia"]

docs = [Document(page_content=t, metadata={"id": id}) for id, t in enumerate(texts)]

In [3]:
faiss_db = FAISS.from_documents(docs, embeddings)

# Search

In [4]:
query = "Những người có tên Quang"
res = faiss_db.similarity_search_with_score(query, k=2)
res

[(Document(page_content='Quang Lê là ca sĩ', metadata={'id': 8}), 0.86171794),
 (Document(page_content='Quang Trung là một vị hoàng đế', metadata={'id': 7}),
  1.0207201)]

# RAG

In [5]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

class MyRunnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            print("invoke")
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter

chat_model = ChatGoogleGenerativeAI(
            model="gemini-pro", 
            temperature=0, 
            google_api_key=conf.load_gemini_token()
        )

template = """\
    Answer the question based only on the following context:\n \
        {context} \
    \nQuestion: {question}"""
prompt = ChatPromptTemplate.from_template(template)


class MemoryLog:

    def __init__(self):
        self.memory = []
        self.state = {}
    
    def log(self, e):
        self.memory.append(e)
        return e

    def store(self, e):
        # self.state.append(e)
        self.state[e['id']] = e
        return e

    def parse(self, e):
        data = {**e}
        data_str = ""
        for c in e['context']:
            data_str += ("\n- " + c.page_content)
        data['context'] = data_str
        return data
    
mem = MemoryLog()

retriever = faiss_db.as_retriever(search_kwargs={"k": 2})

rag = (
    {"context": itemgetter("question") | faiss_db.as_retriever(), "question": itemgetter("question"), "id": itemgetter("id")}
    | RunnableLambda(lambda e: mem.log(e))
    | RunnableLambda(lambda e: mem.parse(e))
    | RunnableLambda(lambda e: mem.store(e))
    | prompt
    | chat_model
    | StrOutputParser()
)


In [13]:
from langsmith.run_helpers import traceable

conf.enable_tracing(project="LEARN")

@traceable(tags=["rag_lcel_multi_input"])
def ask_rag(question):
    a = rag.invoke({"question": question, "id": "hieu-ai"})
    return a

In [14]:
a = ask_rag("Hiếu là ai?")
a

'Họa sĩ'

In [15]:
b = mem.memory[0]
{**b}

{'context': [Document(page_content='Hiếu là họa sĩ', metadata={'id': 3}),
  Document(page_content='Huy là lập trình viên', metadata={'id': 0}),
  Document(page_content='John là người Mỹ', metadata={'id': 1}),
  Document(page_content='Quang Lê là ca sĩ', metadata={'id': 8})],
 'question': 'Hiếu là ai?',
 'id': 'hieu-ai'}

In [16]:
mem.state

{'hieu-ai': {'context': '\n- Hiếu là họa sĩ\n- Huy là lập trình viên\n- John là người Mỹ\n- Quang Lê là ca sĩ',
  'question': 'Hiếu là ai?',
  'id': 'hieu-ai'}}

In [32]:
a = {"topic": RunnablePassthrough()}
type(a)

dict

# References
- https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub
- https://python.langchain.com/docs/integrations/vectorstores/chroma